In [ ]:
# 1) dates should be formatted in excel to yyyy-MM-dd
# 2) data input as excel format

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

In [ ]:
filename = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Input/current_g3x_con_DELETE2.xlsx"
x1 = pd.ExcelFile(filename) # check this out!

In [ ]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_CON_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')
# df_tierMap

In [ ]:
# res = len(x1.sheet_names)
sheetList = x1.sheet_names
sheetList.sort()
# sheetList

In [ ]:
for sheet in sheetList:
    if '1st' in sheet and 'Qtr' in sheet:
        print('Q1 exist')
        df1 = pd.read_excel(filename,sheet_name=sheet,header=1)
        
        df1.columns = df1.columns.str.replace(' ', '_')
        df1.columns = df1.columns.str.replace('#', 'num')
        
        df_current = df1
        
    elif '2nd' in sheet and 'Qtr' in sheet:
        print('Q2 exist')
        df2 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df2.columns = df2.columns.str.replace(' ', '_')
        df2.columns = df2.columns.str.replace('#', 'num')
        
        df_historic1 = df1.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic1 = df_historic1.add_suffix('_Q1')
        
        df_current = df2
        
    elif '3rd' in sheet and 'Qtr' in sheet:
        print('Q3 exist')
        df3 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df3.columns = df3.columns.str.replace(' ', '_')
        df3.columns = df3.columns.str.replace('#', 'num')
        
        df_historic2 = df2.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic2 = df_historic2.add_suffix('_Q2')
        
        df_current = df3
        
    elif '4th' in sheet and 'Qtr' in sheet:
        print('Q4 exist')
        df4 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df4.columns = df4.columns.str.replace(' ', '_')
        df4.columns = df4.columns.str.replace('#', 'num')
            
        df_historic3 = df3.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic3 = df_historic3.add_suffix('_Q3')
        
        df_current = df4

In [ ]:
# df_historic3.columns

In [ ]:
if 'df_historic1' in locals() and 'df_historic2' in locals() and 'df_historic3' in locals():
    df_historic = df_historic1.set_index('G3X_Common_Owner_Q1').join(
        df_historic2.set_index('G3X_Common_Owner_Q2'),how='outer').join(
        df_historic3.set_index('G3X_Common_Owner_Q3'),how='outer')
    
elif 'df_historic1' in locals() and 'df_historic2' in locals():
    print('2 historical quarters')
    df_historic = df_historic1.set_index('G3X_Common_Owner_Q1').join(
        df_historic2.set_index('G3X_Common_Owner_Q2'),how='outer').assign(Incentive_Curr_Tier_Q3=0,
        Net_Shipments_QTD_Q3=0,Units_to_Incnt_Nxt_Tier_Q3=0,Goodyear_Core_Net_Purchases_QTD_Q3=0,
        Kelly_Net_Purchases_QTD_Q3=0,Other_Net_Purchases_QTD_Q3=0,Incentive_Payment_Q3=0)

else:
    print('1 historical quarter')
    df_historic = df_historic1.assign(
        Incentive_Curr_Tier_Q2=0,Net_Shipments_QTD_Q2=0,Units_to_Incnt_Nxt_Tier_Q2=0,Goodyear_Core_Net_Purchases_QTD_Q2=0,
        Kelly_Net_Purchases_QTD_Q2=0,Other_Net_Purchases_QTD_Q2=0,Incentive_Payment_Q2=0,Incentive_Curr_Tier_Q3=0,
        Net_Shipments_QTD_Q3=0,Units_to_Incnt_Nxt_Tier_Q3=0,Goodyear_Core_Net_Purchases_QTD_Q3=0,
        Kelly_Net_Purchases_QTD_Q3=0,Other_Net_Purchases_QTD_Q3=0,Incentive_Payment_Q3=0)

In [ ]:
tup_now = datetime.now().timetuple()
currentYear = datetime.now().year

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df_current.shape[0]):

    singleCusty = df_current.iloc[[ii]].assign(QTDsum=0)
    singleCusty.reset_index(inplace=True,drop=True)

    singleCusty.QTDsum = singleCusty.Goodyear_Core_Net_Purchases_QTD + singleCusty.Kelly_Net_Purchases_QTD + singleCusty.Other_Net_Purchases_QTD
    singleCusty_select = singleCusty[
        ['G3X_CO_Name','num_Locations','Goodyear_Core_Net_Purchases_QTD','Kelly_Net_Purchases_QTD','Other_Net_Purchases_QTD','QTDsum']]
    singleCusty_select = singleCusty_select.rename(index=str, columns={
        'G3X_CO_Name':'Name','num_Locations':'numLocs','Goodyear_Core_Net_Purchases_QTD':'goodyearQTD','Kelly_Net_Purchases_QTD':'kellyQTD','Other_Net_Purchases_QTD':'dunlopQTD'})
    excel_date = singleCusty.Last_Transaction

    date_lt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2)
    tt = date_lt.timetuple()

    if  tt.tm_year == currentYear:

        if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
            if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                print('Leap year')
            else:
                quarter = 1
                daysINqtr = 90           
        elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            subtractDays = 90          
        elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            subtractDays = 90 + 91
        else:
            quarter = 4
            daysINqtr = 92
            subtractDays = 90 + 91 + 92
        
        numDays = tt.tm_yday - subtractDays # get number of days

        estimatedTires = np.floor((singleCusty.QTDsum/(numDays/daysINqtr))).values
        numLocations = singleCusty.num_Locations.values
        
        if   (estimatedTires >= (df_tierMap.iloc[1,4] )) & (estimatedTires <= (df_tierMap.iloc[1,5])):
            tierType = 1
        elif (estimatedTires >= (df_tierMap.iloc[2,4] )) & (estimatedTires <= (df_tierMap.iloc[2,5])):
            tierType = 2
        elif (estimatedTires >= (df_tierMap.iloc[3,4] )) & (estimatedTires <= (df_tierMap.iloc[3,5])):
            tierType = 3
        elif (estimatedTires >= (df_tierMap.iloc[4,4])):
            tierType = 4
        else:
            tierType = 0
        
        if tierType < 4:
            tierUP = tierType + 1
        else:
            tierUP = 4
        
        # floor to underestimate estimated tires
        goodYR_est_tires = np.floor((singleCusty.Goodyear_Core_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        goodYR_est_tires_rebate = goodYR_est_tires * df_tierMap.iloc[tierType,1]
        goodYR_prop = singleCusty_select.goodyearQTD/singleCusty_select.QTDsum
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        goodYR_1up_tires = np.ceil((goodYR_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        goodYR_1up_tires_rebate = goodYR_1up_tires * df_tierMap.iloc[tierUP,1]
        
        kelly_est_tires = np.floor((singleCusty.Kelly_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        kelly_est_tires_rebate = kelly_est_tires * df_tierMap.iloc[tierType,2]
        kelly_prop = singleCusty_select.kellyQTD/singleCusty_select.QTDsum
        kelly_1up_tires = np.ceil((kelly_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        kelly_1up_tires_rebate = kelly_1up_tires * df_tierMap.iloc[tierUP,2]
        
        dunlop_est_tires = np.floor((singleCusty.Other_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        dunlop_est_tires_rebate = dunlop_est_tires * df_tierMap.iloc[tierType,3]
        dunlop_prop = singleCusty_select.dunlopQTD/singleCusty_select.QTDsum
        dunlop_1up_tires = np.ceil((dunlop_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        dunlop_1up_tires_rebate = dunlop_1up_tires * df_tierMap.iloc[tierUP,3]
        
        total_est_tires = goodYR_est_tires + kelly_est_tires + dunlop_est_tires
        total_est_dollars = goodYR_est_tires_rebate+kelly_est_tires_rebate+dunlop_est_tires_rebate
        total_1up_dollars = goodYR_1up_tires_rebate+kelly_1up_tires_rebate+dunlop_1up_tires_rebate
        total_1up_tires = goodYR_1up_tires+kelly_1up_tires+dunlop_1up_tires
        
        if estimatedTires > total_1up_tires:
            goodYR_1up_tires = goodYR_est_tires
            kelly_1up_tires = kelly_est_tires
            dunlop_1up_tires = dunlop_est_tires
            total_1up_tires = estimatedTires
            
            goodYR_1up_tires_rebate = goodYR_est_tires_rebate
            kelly_1up_tires_rebate = kelly_est_tires_rebate
            dunlop_1up_tires_rebate = dunlop_est_tires_rebate
            total_1up_dollars = total_est_dollars

        range_dollars_ll = ((total_1up_tires*df_tierMap.iloc[tierUP,3]))#.values
        range_dollars_ul = ((total_1up_tires*df_tierMap.iloc[tierUP,1]))#.values
        
        tempData = pd.DataFrame(
            {'estimated_tier':tierType,
             'GY_est_tires':goodYR_est_tires, 'Kelly_est_tires':kelly_est_tires, 'Dunlop_est_tires':dunlop_est_tires, 'current_est_tires':total_est_tires*numLocations,
             'GY_est_rebate':goodYR_est_tires_rebate, 'Kelly_est_rebate':kelly_est_tires_rebate, 'Dunlop_est_rebate':dunlop_est_tires_rebate,
             'total_est_rebate': total_est_dollars,'Up_tier':tierUP,
             'GY_up_tires':goodYR_1up_tires, 'Kelly_up_tires':kelly_1up_tires, 'Dunlop_up_tires':dunlop_1up_tires, 'total_up_tires':total_1up_tires,
             'GY_up_rebate':goodYR_1up_tires_rebate, 'Kelly_up_rebate':kelly_1up_tires_rebate, 'Dunlop_up_rebate':dunlop_1up_tires_rebate,
             'total_up_rebate':total_1up_dollars,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
        range_dollars_ll = (df_tierMap.iloc[1,3] * df_tierMap.iloc[1,4])
        range_dollars_ul = (df_tierMap.iloc[1,1] * df_tierMap.iloc[1,4]) 
        tempData = pd.DataFrame(
            {'estimated_tier':0,
             'GY_est_tires':0, 'Kelly_est_tires':0, 'Dunlop_est_tires':0, 'current_est_tires':0, 
             'GY_est_rebate':0, 'Kelly_est_rebate':0, 'Dunlop_est_rebate':0,
             'total_est_rebate': 0, 'Up_tier':1,
             'GY_up_tires':0, 'Kelly_up_tires':0, 'Dunlop_up_tires':0, 'total_up_tires':0,
             'GY_up_rebate':0, 'Kelly_up_rebate':0, 'Dunlop_up_rebate':0,
             'total_up_rebate':0,
             'range_lower':[range_dollars_ll], 'range_upper':[range_dollars_ul],'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df_current], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_G3X_CON.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_CON_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/csv_quarter_history/'
writeFilename = path + 'CMP_G3X_CON_historic.csv'
df_historic.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_historic_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_CON_historic_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
df_historic.to_csv(writeFilename_archive, index=False)